# CalTech 101

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting

%reload_ext autoreload
%autoreload 2
%matplotlib inline

Using fastai lib

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import time
from sklearn.model_selection import StratifiedKFold

In [3]:
# Uncomment the below if you need to reset your precomputed activations
!rm -rf {PATH}tmp

Set model parameters:

In [4]:
PATH = "data/caltech/direct/"
SZ=224 #resnet restriction
ARCH = resnet50
arch = ARCH

TFMS = tfms_from_model(ARCH, 
                       SZ, 
                       aug_tfms=transforms_side_on, 
                       max_zoom=1.2)


In [5]:
def loadImages(filenames):
  imgs = []
  
  for j in range(0, len(filenames)):
    fn = str(PATH+filenames[j])
    try:
        im = cv2.imread(str(fn))
        if im is None: raise OSError(f'File not recognized by opencv: {fn}')
    except Exception as e:
        raise OSError('Error handling image at: {}'.format(fn)) from e
    im = cv2.resize(im, (SZ,SZ))
    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB).astype(np.float32)/255)
  return imgs


In [6]:
def getXY():
    data = ImageClassifierData.from_paths(path=PATH, tfms=TFMS)

    train_files = data.trn_ds.fnames
    train_categories = data.trn_y
    train = data.trn_ds
    X = np.asarray(loadImages(train_files))
    Y = np.asarray(train_categories)
    return X, Y

In [7]:
def getFolds(X, Y):
  dummy = ImageClassifierData.from_paths(path=PATH, tfms=TFMS, trn_name='test')
  tst = np.asarray(loadImages(dummy.trn_ds.fnames))
  tst_ctg = dummy.trn_y
  skf = StratifiedKFold(n_splits=5)
  i =0
  data = []
  index = []
  for train_index, val_index in skf.split(X, Y):
      index.append((train_index, val_index))
      X_train = X[train_index]
      Y_train = Y[train_index]
      X_valid = X[val_index]
      Y_valid = Y[val_index]
      trn = (X_train, Y_train)
      val = (X_valid, Y_valid)
      data.append(ImageClassifierData.from_arrays(path=PATH, bs=4,
                                                  trn=trn, val=val, tfms=TFMS, test=(tst, tst_ctg)))
      i+=1
  return data, index

In [8]:
def trainModels(data):
    m=[]  
    for i in range(len(data)):
        m.append(ConvLearner.pretrained(arch, data[i], precompute=False))
    return m


In [9]:
def runSingleRate(models):
  print ('running single rate')
  start = time.time()
  for i in range(len(data)):
      startmodel = time.time()
      learn = models[i]
      learn.fit(0.012, 30)
      endmodel = time.time()
      duration = endmodel - startmodel
      print('model{} took {}s'.format(i, duration))
  finish = time.time()
  duration = finish - start
  print('single rate tool {}s in total'.format(duration))


In [10]:
def runCyclicalRate(models):
  print ('running cyclical rate')
  start = time.time()
  for i in range(len(data)):
      startmodel = time.time()
      learn = models[i]
      learn.fit(0.012, 5, cycle_len=1, cycle_mult=2)
      endmodel = time.time()
      duration = endmodel - startmodel
      print ('model {} took {}s'.format(i, duration))
  end = time.time()
  duration = end - start
  print('Cyclical rate tool {}s in total'.format(duration))  

In [11]:
def predictImages(models):
    print ('predicting images')
    preds=[]
    start = time.time()  
    for i in range(len(data)):
        s = time.time()
        learn=models[i]
        log_preds, y=learn.TTA(is_test=True)
        p=np.mean(np.exp(log_preds), 0)
        preds.append(p)
        e = time.time()
        d = e-s
        print ('predicting for model {} took {}s'.format(i,d))
    mean=np.mean(preds, 0)
    end =  time.time()
    duration = end - start
    print('Predicting took {}s'.format(duration))
    return mean, y

In [12]:
X, Y = getXY()

In [13]:
data, indexes = getFolds(X, Y)

In [14]:
for j in range(3):
  m1 = trainModels(data)
  runSingleRate(m1)
  preds, y =predictImages(m1)
  result=accuracy_np(preds, y)
  print('result single rate {}: {}'.format(j, result))

running single rate


HBox(children=(IntProgress(value=0, description='Epoch', max=30), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      3.97485    2.113501   0.490196  
    1      2.545837   1.555797   0.627451                   
    2      1.679703   1.283281   0.637255                   
    3      1.231134   1.119385   0.686275                   
    4      1.037233   0.995715   0.676471                   
    5      0.836867   1.048589   0.676471                    
    6      0.702712   0.989124   0.696078                    
    7      0.607287   0.945399   0.72549                     
    8      0.53815    1.019549   0.754902                    
    9      0.385784   0.979393   0.72549                     
    10     0.48417    0.96018    0.745098                    
    11     0.372646   0.878391   0.784314                    
    12     0.315005   0.894089   0.803922                    
    13     0.308752   0.805243   0.803922                    
    14     0.3534     0.873264   0.794118                    
    15     0.384271   0.818726 

HBox(children=(IntProgress(value=0, description='Epoch', max=30), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      4.16396    1.982196   0.568627  
    1      2.444928   1.279033   0.637255                   
    2      1.586206   0.999397   0.72549                    
    3      1.17117    0.921962   0.764706                   
    4      1.036476   0.874122   0.764706                   
    5      0.810611   0.712514   0.803922                    
    6      0.671157   0.789794   0.754902                    
    7      0.600166   0.673097   0.794118                    
    8      0.496335   0.823057   0.754902                    
    9      0.411746   0.832365   0.764706                    
    10     0.49641    0.83834    0.754902                    
    11     0.436557   0.829302   0.784314                    
    12     0.445503   0.694177   0.784314                    
    13     0.343008   0.746499   0.803922                    
    14     0.361419   0.852757   0.803922                    
    15     0.304785   0.768148 

...

epoch      trn_loss   val_loss   accuracy                   
    0      4.106016   1.886669   0.5       
    1      2.348127   1.295528   0.647059                   
    2      1.675203   1.182366   0.676471                   
    3      1.052876   1.196384   0.77451                    
    4      0.969611   1.070194   0.745098                    
    5      0.758114   1.170468   0.745098                    
    6      0.626766   1.203603   0.735294                    
    7      0.532867   1.191791   0.72549                     
    8      0.482956   1.101795   0.754902                    
    9      0.449366   1.08666    0.735294                    
    10     0.380729   1.10935    0.735294                    
    11     0.41953    1.142253   0.745098                   
    12     0.26599    0.965277   0.745098                    
    13     0.423809   1.120536   0.754902                    
    14     0.356505   1.141949   0.754902                    
    15     0.363853   1.386425 

HBox(children=(IntProgress(value=0, description='Epoch', max=30), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      4.081292   1.885801   0.627451  
    1      2.645472   1.336645   0.676471                   
    2      1.813795   0.998232   0.745098                   
    3      1.222862   0.921778   0.745098                   
    4      0.959396   0.884682   0.794118                    
    5      0.678005   0.878026   0.813725                    
    6      0.616806   0.760799   0.803922                    
    7      0.57175    0.877453   0.794118                    
    8      0.525398   0.840676   0.784314                    
    9      0.459921   1.024579   0.745098                   
    10     0.513715   0.767045   0.833333                    
    11     0.495765   0.885588   0.813725                    
    12     0.433034   0.931835   0.803922                    
    13     0.352999   1.012414   0.803922                    
    14     0.362652   0.915508   0.803922                    
    15     0.245144   0.858321 

HBox(children=(IntProgress(value=0, description='Epoch', max=30), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      4.232193   1.747836   0.529412  
    1      2.56065    1.157258   0.686275                   
    2      1.500468   0.891599   0.715686                   
    3      1.218283   0.838116   0.784314                   
    4      1.107      0.663489   0.803922                   
    5      0.816489   0.72182    0.823529                    
    6      0.622822   0.682653   0.794118                    
    7      0.48859    0.666335   0.77451                     
    8      0.440576   0.79045    0.784314                    
    9      0.453373   0.836865   0.803922                    
    10     0.452004   0.778325   0.794118                    
    11     0.462044   0.820845   0.813725                    
    12     0.32694    0.741536   0.784314                    
    13     0.216342   0.813551   0.794118                    
    14     0.328908   0.652689   0.803922                    
    15     0.422735   0.691282 

In [15]:
%%time
for j in range(3):
  m2 = trainModels(data)
  runCyclicalRate(m2)
  preds, y =predictImages(m2)
  result=accuracy_np(preds, y)
  print('result cyclical rate {}: {}'.format(j, result))

running cyclical rate


HBox(children=(IntProgress(value=0, description='Epoch', max=31), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      4.264588   2.659214   0.392157  
    1      2.909347   1.484593   0.598039                   
    2      1.654718   1.397201   0.705882                   
    3      1.590463   1.328499   0.666667                   
    4      1.305283   1.066216   0.705882                   
    5      0.84367    1.001179   0.72549                     
    6      0.58077    0.934518   0.745098                    
    7      0.738393   1.067819   0.764706                    
    8      0.648924   1.033141   0.715686                    
    9      0.538097   1.018523   0.72549                     
    10     0.447406   0.95952    0.745098                    
    11     0.375883   0.903955   0.77451                     
    12     0.325007   0.885891   0.77451                     
    13     0.277534   0.900527   0.745098                    
    14     0.271005   0.919932   0.754902                    
    15     0.379271   1.172605 

HBox(children=(IntProgress(value=0, description='Epoch', max=31), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      4.30684    2.670974   0.401961  
    1      2.882864   1.352913   0.72549                    
    2      1.777557   1.187456   0.676471                   
    3      1.629147   1.097313   0.705882                   
    4      1.030705   0.94167    0.77451                    
    5      0.638014   0.810116   0.784314                    
    6      0.519541   0.820977   0.784314                    
    7      0.74563    0.863293   0.784314                    
    8      0.787953   0.732626   0.794118                    
    9      0.722886   0.648313   0.764706                    
    10     0.44915    0.799645   0.754902                    
    11     0.348804   0.731754   0.794118                    
    12     0.262096   0.682846   0.803922                    
    13     0.328908   0.609848   0.803922                    
    14     0.278094   0.632556   0.784314                    
    15     0.396166   0.819711 

...

epoch      trn_loss   val_loss   accuracy                   
    0      4.159198   2.749343   0.392157  
    1      2.741774   1.677563   0.578431                   
    2      1.694134   1.433344   0.686275                   
    3      1.62402    1.374581   0.715686                   
    4      1.059675   1.125942   0.735294                   
    5      0.764514   1.094847   0.745098                    
    6      0.591777   1.049923   0.745098                    
    7      0.720631   1.246648   0.72549                     
    8      0.712982   1.250402   0.72549                     
    9      0.6151     1.150212   0.72549                     
    10     0.4911     1.239692   0.745098                    
    11     0.399527   1.196419   0.764706                   
    12     0.272836   1.102992   0.764706                    
    13     0.22358    1.129807   0.784314                    
    14     0.268718   1.04508    0.784314                    
    15     0.360391   1.360321  

HBox(children=(IntProgress(value=0, description='Epoch', max=31), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      4.333248   2.664396   0.460784  
    1      2.860782   1.470404   0.676471                   
    2      1.662286   1.288387   0.656863                   
    3      1.47541    1.019157   0.72549                    
    4      1.188742   0.887136   0.764706                   
    5      0.70292    0.807074   0.803922                    
    6      0.565062   0.801142   0.794118                    
    7      0.928013   0.745311   0.803922                    
    8      0.67818    0.776846   0.813725                    
    9      0.649768   0.890076   0.813725                    
    10     0.591418   0.713603   0.843137                    
    11     0.383756   0.75094    0.813725                    
    12     0.298982   0.682767   0.823529                    
    13     0.279557   0.657512   0.833333                   
    14     0.305767   0.637801   0.833333                    
    15     0.349912   0.625346  

HBox(children=(IntProgress(value=0, description='Epoch', max=31), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      4.238073   2.556891   0.421569  
    1      2.916721   1.47966    0.637255                   
    2      1.698196   1.336704   0.696078                  
    3      1.535782   1.038236   0.72549                    
    4      1.043991   0.877598   0.735294                   
    5      0.661891   0.822405   0.754902                    
    6      0.573584   0.851634   0.754902                    
    7      0.73582    0.902022   0.735294                    
    8      0.771504   0.960146   0.715686                    
    9      0.615516   0.971147   0.72549                     
    10     0.6306     0.813671   0.803922                    
    11     0.495099   0.818919   0.764706                    
    12     0.296199   0.758648   0.784314                    
    13     0.341582   0.74102    0.794118                    
    14     0.299312   0.747589   0.784314                    
    15     0.415912   0.886974  